#  Работа с категориальными признаками

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
!pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn"

geopandas==0.13.2
lightgbm==4.1.0
numpy==1.25.2
pandas==1.5.3
pandas-datareader==0.10.0
pandas-gbq==0.19.2
pandas-stubs==1.5.3.230304
scikit-learn==1.2.2
sklearn-pandas==2.2.0


## Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
train = pd.read_parquet('/content/drive/MyDrive/Purple_Hack/train_data.pqt') # Подгружаем файл тренировочных данных
test = pd.read_parquet('/content/drive/MyDrive/Purple_Hack/test_data.pqt') # Подгружаем файл тестовых данных

# Получим список категориальных признаков

In [ ]:
object_columns_train = [s for s in train.columns if train[s].dtypes == 'object']
print('{} - категориальные столбцы датасета, всего их - {} штук'.format(object_columns_train, len(object_columns_train)))

['date', 'channel_code', 'city', 'city_type', 'index_city_code', 'ogrn_month', 'ogrn_year', 'okved', 'segment', 'start_cluster', 'end_cluster'] - категориальные столбцы датасета, всего их - 11 штук


Получим список столбцов, которые есть в train, но которых нет в test

In [ ]:
for col_train in train.columns:
  if col_train not in test.columns:
    print('Признака {} - нет в тестовых данных'.format(col_train))

Признака end_cluster - нет в тестовых данных


Поскольку для работы с пропусками в файле test необходимо иметь полное представление о данных, создадим новый признак sample - отделяющий тестовые данные от тренировочных. Также для объединения данных для заполнения признаков создадим признак __end_cluster__ в тестовом наборе, чтобы избежать проблем при объединении.

In [ ]:
test['end_cluster'] = 0

In [ ]:
test['sample'] = 1  # Тестовые данные
train['sample'] = 0 # Тренировочные данные

data = pd.concat([test, train], ignore_index=True).reset_index(drop=True)

In [ ]:
null_data = data.isnull()
cols_with_null = null_data[null_data > 0].sum()

In [ ]:
columns_with_nan = list(cols_with_null[cols_with_null>0].index)

In [ ]:
print(columns_with_nan)

['balance_amt_avg', 'balance_amt_max', 'balance_amt_min', 'balance_amt_day_avg', 'channel_code', 'city', 'city_type', 'index_city_code', 'ogrn_days_end_month', 'ogrn_days_end_quarter', 'ogrn_month', 'ogrn_year', 'ft_registration_date', 'max_founderpres', 'min_founderpres', 'ogrn_exist_months', 'okved', 'segment', 'sum_of_paym_2m', 'sum_of_paym_6m', 'sum_of_paym_1y', 'cnt_a_oper_1m', 'cnt_b_oper_1m', 'cnt_c_oper_1m', 'cnt_deb_d_oper_1m', 'cnt_cred_d_oper_1m', 'cnt_deb_e_oper_1m', 'cnt_days_deb_e_oper_1m', 'cnt_cred_e_oper_1m', 'cnt_days_cred_e_oper_1m', 'cnt_deb_f_oper_1m', 'cnt_days_deb_f_oper_1m', 'cnt_cred_f_oper_1m', 'cnt_days_cred_f_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_days_deb_g_oper_1m', 'cnt_cred_g_oper_1m', 'cnt_days_cred_g_oper_1m', 'cnt_deb_h_oper_1m', 'cnt_days_deb_h_oper_1m', 'cnt_cred_h_oper_1m', 'cnt_days_cred_h_oper_1m', 'cnt_a_oper_3m', 'cnt_b_oper_3m', 'cnt_c_oper_3m', 'cnt_deb_d_oper_3m', 'cnt_cred_d_oper_3m', 'cnt_deb_e_oper_3m', 'cnt_days_deb_e_oper_3m', 'cnt_cred_e_

Получим список тех категориальных столбцов, в которых имеются пропуски


In [ ]:
cat_cols_with_null = list()
for col in object_columns_train:
  if col in columns_with_nan:
    cat_cols_with_null.append(col)
  else:
    continue

print('Список категориальных признаков с пропусками - {}, число таких столбцов - {}'.format(cat_cols_with_null, len(cat_cols_with_null)))

Список категориальных признаков с пропусками - ['channel_code', 'city', 'city_type', 'index_city_code', 'ogrn_month', 'ogrn_year', 'okved', 'segment', 'start_cluster'], число таких столбцов - 9


Выделим ту часть данных, в категориальных признаках которой есть пропуски. Будем отдельно работать с этими данными с пропусками, а после внесем полученные изменения в изначальный датасет.

In [ ]:
data_cat_null = data[cat_cols_with_null].copy()

In [ ]:
display(data_cat_null)

,channel_code,city,city_type,index_city_code,ogrn_month,ogrn_year,okved,segment,start_cluster
0,channel_code_12,city_14,city_type_0,None,ogrn_month_8,ogrn_year_11,okved_0,segment_3,{α}
1,channel_code_12,city_14,city_type_0,None,ogrn_month_8,ogrn_year_11,okved_0,segment_3,{α}
2,channel_code_12,city_14,city_type_0,None,ogrn_month_8,ogrn_year_11,okved_0,segment_3,None
3,channel_code_9,city_76,city_type_0,None,ogrn_month_10,ogrn_year_11,okved_6,segment_3,{α}
4,channel_code_9,city_76,city_type_0,None,ogrn_month_10,ogrn_year_11,okved_6,segment_3,{α}
...,...,...,...,...,...,...,...,...,...
890115,channel_code_9,city_14,city_type_0,None,ogrn_month_6,ogrn_year_10,okved_5,segment_3,{}
890116,channel_code_9,city_14,city_type_0,None,ogrn_month_6,ogrn_year_10,okved_5,segment_3,{α}
890117,channel_code_14,city_1876,city_type_0,index_city_code_195,ogrn_month_2,ogrn_year_17,okved_31,segment_3,{}
890118,channel_code_14,city_1876,city_type_0,index_city_code_195,ogrn_month_2,ogrn_year_17,okved_31,segment_3,{}


In [ ]:
data_cat_null.describe()

,channel_code,city,city_type,index_city_code,ogrn_month,ogrn_year,okved,segment,start_cluster
count,827242,751164,750784,408435,844369,844369,843748,845949,790120
unique,45,8917,7931,233,12,21,88,4,17
top,channel_code_8,city_0,city_type_0,index_city_code_46,ogrn_month_0,ogrn_year_4,okved_5,segment_3,{α}
freq,152175,125034,670615,14974,80029,139179,129948,508446,491192


Заменим значения в каждом столбце на моду


In [ ]:
fill_data1 = data.copy()

values = {
    'channel_code': data['channel_code'].mode(),
    'city': data['city'].mode(),
    'city_type': data['city_type'].mode(),
    'index_city_code': data['city_type'].mode(),
    'ogrn_month': data['ogrn_month'].mode(),
    'ogrn_year': data['ogrn_year'].mode(),
    'okved': data['okved'].mode(),
    'segment': data['segment'].mode(),
    'start_cluster': data['start_cluster'].mode()
}

fill_data1 = fill_data1.fillna(values)
fill_data1.isnull().mean()

id                         0.000000
date                       0.000000
balance_amt_avg            0.106795
balance_amt_max            0.106795
balance_amt_min            0.106795
                             ...   
cnt_cred_h_oper_3m         0.275040
cnt_days_cred_h_oper_3m    0.275040
start_cluster              0.112344
end_cluster                0.000000
sample                     0.000000
Length: 94, dtype: float64

Разделим наш датасет обратно и получим новые данные для нашей модели

In [ ]:
train = fill_data1.query('sample == 0').drop(['sample'], axis=1) # Восстанавливаем train
test = fill_data1.query('sample == 1').drop(['sample', 'end_cluster'], axis=1) # Восстанавливаем test

In [ ]:
train.to_parquet('train1.pqt', index=False)
test.to_parquet('test1.pqt', index=False)